Да, это ноутбук с лекции. Далее в нём будут изменения, но незначительные.

In [10]:
import torch
from src_allennlp.model import SimpleTagger
from src_allennlp.reader import RuPosReader
from allennlp.data.vocabulary import Vocabulary
from allennlp.common import Params
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding, TokenCharactersEncoder
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


В ридер добавлены символьные эмбеддинги.</br>
Модель теперь выглядит так:</br>
Embedding -> Highway -> BiLSTM -> FeedForward -> Linear -> CRF</br>
Вообще, в задачах теггинга BiLSTM+CRF - это самая классика.

In [11]:
reader = RuPosReader()
dataset = reader.read('data/train.csv')
vocab = Vocabulary.from_instances(dataset, pretrained_files={'tokens':'data/cc.ru.300.vec'}, 
                                  only_include_pretrained_words=False)

/usr/local/lib/python3.6/dist-packages/allennlp/data/token_indexers/token_characters_indexer.py:56: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)
48171it [00:04, 10988.71it/s]
100%|██████████| 2000000/2000000 [00:33<00:00, 58904.49it/s] 


А вот и эмбеддинги (пословные + символьные):

In [12]:
word_emb = 300
char_emb = 20
char_dim = 50
hidden_dim = 300

charemb = Embedding(vocab.get_vocab_size('token_characters'), char_emb)
charrnn = PytorchSeq2VecWrapper(torch.nn.LSTM(char_emb, char_dim, batch_first=True, bidirectional=True))
charlstm = TokenCharactersEncoder(charemb, charrnn)
params = Params({"pretrained_file": 'data/cc.ru.300.vec', "embedding_dim": word_emb, "trainable": False})
embedder = BasicTextFieldEmbedder({"tokens": Embedding.from_params(vocab, params),
                                   "chars": charlstm})
encoder = PytorchSeq2SeqWrapper(torch.nn.LSTM(embedder.get_output_dim(), hidden_dim, batch_first=True, bidirectional=True, num_layers=2))
model = SimpleTagger(vocab, embedder, encoder)

100%|██████████| 2000000/2000000 [00:26<00:00, 76863.23it/s] 


In [15]:
data_size = len(dataset)

train_dataset = dataset[:int(data_size * 0.96)]
val_dataset = dataset[int(data_size * 0.96):int(data_size * 0.99)]
test_dataset = dataset[int(data_size * 0.99):]

In [16]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

46244
1445
482


Дальше вообще ничего не трогаю. Только тестирую, не на валидационной выборке, а на отдельной тестовой.

Распиливаем датасет, слайсы здесь работают нормально

In [17]:
device = torch.device('cuda')
model.to(device)
optimizer = torch.optim.Adam(model.parameters())
iterator = BucketIterator(batch_size=256, sorting_keys=[("tokens", "num_tokens")], biggest_batch_first=True)
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=val_dataset,
                  patience=5,
                  num_epochs=20,
                  cuda_device=0,
                  validation_metric="+fscore")

Переносим модель на видеокарту, задаём оптимизатор, итератор с индексированием по словарю и сам тренер. У тренера есть куча удобных параметров, например, сколько эпох учить, через сколько эпох остановиться, если таргетная метрика перестаёт расти на валидации (`patience`) и собственно сама таргетная метрика, по которой будет выбираться лучшая эпоха

In [18]:
trainer.train()

accuracy: 0.6325, precision: 0.6325, recall: 0.6325, fscore: 0.6325, loss: 5162.2751 ||: 100%|██████████| 181/181 [00:35<00:00,  5.17it/s]
accuracy: 0.8877, precision: 0.8877, recall: 0.8877, fscore: 0.8877, loss: 1407.4273 ||: 100%|██████████| 6/6 [00:00<00:00,  6.97it/s]
accuracy: 0.9294, precision: 0.9294, recall: 0.9294, fscore: 0.9294, loss: 999.1220 ||: 100%|██████████| 181/181 [00:25<00:00,  7.21it/s] 
accuracy: 0.9573, precision: 0.9573, recall: 0.9573, fscore: 0.9573, loss: 558.9244 ||: 100%|██████████| 6/6 [00:00<00:00, 10.30it/s]
accuracy: 0.9635, precision: 0.9635, recall: 0.9635, fscore: 0.9635, loss: 522.0363 ||: 100%|██████████| 181/181 [00:25<00:00,  7.17it/s]
accuracy: 0.9727, precision: 0.9727, recall: 0.9727, fscore: 0.9727, loss: 360.7397 ||: 100%|██████████| 6/6 [00:00<00:00, 11.07it/s]
accuracy: 0.9730, precision: 0.9730, recall: 0.9730, fscore: 0.9730, loss: 378.4503 ||: 100%|██████████| 181/181 [00:25<00:00,  7.08it/s]
accuracy: 0.9765, precision: 0.9765, recall

{'best_epoch': 17,
 'best_validation_accuracy': 0.9872463081418306,
 'best_validation_fscore': 0.9872463345527649,
 'best_validation_loss': 182.6574452718099,
 'best_validation_precision': 0.9872463345527649,
 'best_validation_recall': 0.9872463345527649,
 'epoch': 19,
 'peak_cpu_memory_MB': 4537.096,
 'peak_gpu_0_memory_MB': 3085,
 'training_accuracy': 0.9964818344405639,
 'training_cpu_memory_MB': 4537.096,
 'training_duration': '0:08:48.473534',
 'training_epochs': 19,
 'training_fscore': 0.9964818358421326,
 'training_gpu_0_memory_MB': 3085,
 'training_loss': 43.82599611440416,
 'training_precision': 0.9964818358421326,
 'training_recall': 0.9964818358421326,
 'training_start_epoch': 0,
 'validation_accuracy': 0.9867724867724867,
 'validation_fscore': 0.9867724776268005,
 'validation_loss': 218.86179733276367,
 'validation_precision': 0.9867724776268005,
 'validation_recall': 0.9867724776268005}

In [38]:
model.eval()
with torch.no_grad():
    model.forward_on_instances(test_dataset)
    print(model.get_metrics())

{'accuracy': 0.9845385160122747, 'precision': 0.9845384955406189, 'recall': 0.9845384955406189, 'fscore': 0.9845384955406189}


In [22]:
model.eval()
with torch.no_grad():
    labels =  model.forward_on_instance(test_dataset[1])['labels']

for token, label in zip(test_dataset[1]['tokens'].tokens,labels):
    print(token, label)

Излишне ADJ
говорить VERB
о ADP
том PRON
, PUNCT
что SCONJ
, PUNCT
оказавшись VERB
снова ADV
в ADP
камере NOUN
, PUNCT
Шура PROPN
мысленно ADV
уже ADV
прощалась VERB
с ADP
жизнью NOUN
. PUNCT
